In [1]:
import os
import shutil
import warnings
from pathlib import Path
import pathlib
import pandas as pd
from helper import load_data
from helper import get_root
from helper import save_data
from transform_fcts import user_to_id

In [2]:
def upper_to_lower(df):
    df = df.str.lower()
    return df

In [3]:
def make_u_to_ue(df):
    df = df.str.replace('u','ue')
    return df

In [4]:
def load_data_semi(filename: str) -> pd.DataFrame:
    """Load any data csv file from the data folder

    Args:
        filename (str): Name of the csv file to load

    Returns:
        pd.DataFrame: Dataframe of all file data

    """
    data_file = os.path.join(get_root(), "data", filename)
    data_df = pd.read_csv(data_file, delimiter=';')

    return data_df

In [7]:
def generate_data_response():
    df = load_data_semi('data_ubung_response.csv')
    df = df[df["User"].str.contains('HE')]
    user_id = df["User"].str[-3:].astype(int)
    df["User"] = user_id - min(user_id)
    df.dropna(axis=0, inplace=True)
    df.drop('Code', axis=1, inplace=True)
    df.drop('Date/Time', axis=1, inplace=True)
    df.drop('Aufgabenniveau', axis=1, inplace=True)
    df['Task'] = make_u_to_ue(upper_to_lower(df['Task']))

    save_data(df, 'data_mikro_response')
    return df

In [ ]:
df_res = generate_data_response()
#df_res

In [17]:
df_res[df_res['User']==0][df_res['Task']=='ks1c1ue3_1']['ResponsTask'].iloc[0]

C:\Users\Lukas\AppData\Local\Temp/ipykernel_17688/1653907.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_res[df_res['User']==0][df_res['Task']=='ks1c1ue3_1']['ResponsTask'].iloc[0]


'twr'

In [18]:
def generate_data_exercise():
    df = load_data('data_trafo.csv')
    df.drop('Unnamed: 0',axis=1, inplace=True)
    df.drop(df[df['Category'] == -1].index, inplace=True)
    df.drop(df[df['Label'] == 'Tour'].index, inplace=True)
    #df.drop(df[df['Label'] == 'Startseite'].index, inplace=True)
    #df.drop(df[df['Label'] == 'Übersicht'].index, inplace=True)
    df.drop(df[df['Label'] == 'Rückmeldungen'][df['TestResQual'] == -1].index, inplace=True)

    #df_rt = load_data('data_mikro_response.csv')
    df_rt = generate_data_response()
    df.reset_index(inplace=True)
    df_user = df.groupby('User')
    df_mikro = []
    for user in df_user:
        index_tests = user[1].loc[user[1]['LearnType'] == 2].index

        for index in index_tests:
            df_mikro.append([user[0],
                            df.iloc[index - 1]['Category'],
                            df.iloc[index - 1]['LearnType'],
                            df.iloc[index]['Category'],
                            df.iloc[index]['LearnType'],
                            df.iloc[index]['difficulty'],
                            df_rt[df_res['User']==user[0]]
                             [df_rt['Task']==df.iloc[index]['Label']]['ResponsTask'].iloc[0],
                            df.iloc[index + 1]['Category'],
                            df.iloc[index + 1]['LearnType']])

    df_mikro = pd.DataFrame(df_mikro, columns=['User',
                                               'PrevActCat',
                                               'PrevActLT',
                                               'ExerciseCat',
                                               'ExerciseLT',
                                               'Difficulty',
                                               'ResponseTask',
                                               'NextActCat',
                                               'NextActLT'])

    return df_mikro

In [19]:
df_ex = generate_data_exercise()

C:\Users\Lukas\AppData\Local\Temp/ipykernel_17688/1000858769.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.drop(df[df['Label'] == 'Rückmeldungen'][df['TestResQual'] == -1].index, inplace=True)
C:\Users\Lukas\AppData\Local\Temp/ipykernel_17688/1000858769.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_rt[df_res['User']==user[0]]


IndexError: single positional indexer is out-of-bounds

In [20]:
df_ex

,User,PrevActCat,PrevActLT,ExerciseCat,ExerciseLT,Difficulty,ResponseTask,NextActCat,NextActLT
0,0,0,1,0,2,0,"161 twr Name: ResponsTask, dtype: object",0,6
1,0,1,1,1,2,0,"162 r Name: ResponsTask, dtype: object",1,1
2,0,1,1,1,2,0,"162 r Name: ResponsTask, dtype: object",1,2
3,0,1,2,1,2,0,"162 r Name: ResponsTask, dtype: object",1,6
4,0,0,1,0,2,0,"161 twr Name: ResponsTask, dtype: object",0,2
...,...,...,...,...,...,...,...,...,...
532,43,1,1,1,2,2,"Series([], Name: ResponsTask, dtype: object)",1,6
533,45,0,1,0,2,0,"Series([], Name: ResponsTask, dtype: object)",0,2
534,45,0,2,0,2,0,"Series([], Name: ResponsTask, dtype: object)",0,1
535,46,0,1,0,2,0,"Series([], Name: ResponsTask, dtype: object)",0,1


In [8]:
df_ex.drop(['User'], inplace=True, axis=1)
df_ex.groupby(df_ex.columns.tolist()).size().reset_index().rename(columns={0:'count traces'})

,PrevActCat,PrevActLT,ExerciseCat,ExerciseLT,Difficulty,NextActCat,NextActLT,count traces
0,0,1,0,2,0,0,1,32
1,0,1,0,2,0,0,2,21
2,0,1,0,2,0,0,6,1
3,0,1,0,2,0,1,1,2
4,0,1,0,2,1,0,1,31
...,...,...,...,...,...,...,...,...
77,5,1,5,2,1,5,1,3
78,5,1,5,2,1,5,2,1
79,5,2,5,2,1,5,2,1
80,5,2,5,2,2,5,1,1


In [9]:
save_data(df_ex, 'data_mikro_exer_traces_82')

In [19]:
diff_next = df_ex['NextActCat'] == df_ex['ExerciseCat']
next_as_int = [0 if next else 1 for next in diff_next]

#df_test_conv = df_test.copy()
diff_prev_exer = df_ex['PrevActCat'] - df_ex['ExerciseCat']
#diff_next_test = int(not(df_test['NextAct'] == df_test['Test']))
df_ex['PrevActCat'] = diff_prev_exer
#df_ex['ExerciseCat'] = diff_prev_exer
df_ex['NextActCat'] = next_as_int
df_ex

,User,PrevActCat,PrevActLT,ExerciseCat,ExerciseLT,Difficulty,NextActCat,NextActLT
0,0,0,1,0,2,0,0,6
1,0,0,1,1,2,0,0,1
2,0,0,1,1,2,0,0,2
3,0,0,2,1,2,0,0,6
4,0,0,1,0,2,0,0,2
...,...,...,...,...,...,...,...,...
532,43,0,1,1,2,2,0,6
533,45,0,1,0,2,0,0,2
534,45,0,2,0,2,0,0,1
535,46,0,1,0,2,0,0,1


In [20]:
df_ex.drop(['User'], inplace=True, axis=1)
df_ex.groupby(df_ex.columns.tolist()).size().reset_index().rename(columns={0:'count traces'})

,PrevActCat,PrevActLT,ExerciseCat,ExerciseLT,Difficulty,NextActCat,NextActLT,count traces
0,0,1,0,2,0,0,1,32
1,0,1,0,2,0,0,2,21
2,0,1,0,2,0,0,6,1
3,0,1,0,2,0,1,1,2
4,0,1,0,2,1,0,1,31
...,...,...,...,...,...,...,...,...
77,0,2,4,2,2,0,6,1
78,0,2,5,2,1,0,2,1
79,0,2,5,2,2,0,1,1
80,0,2,5,2,2,0,2,1
